## **Descripción del ejercicio**

Has recibido una tarea analítica de una tienda en línea internacional. Tus predecesores no consiguieron completarla: lanzaron una prueba A/B y luego abandonaron (para iniciar una granja de sandías en Brasil). Solo dejaron las especificaciones técnicas y los resultados de las pruebas.

### Descripción técnica

- Nombre de la prueba: `recommender_system_test`
- Grupos: А (control), B (nuevo embudo de pago)
- Launch date: 2020-12-07
- Fecha en la que dejaron de aceptar nuevos usuarios: 2020-12-21
- Fecha de finalización: 2021-01-01
- Audiencia: 15% de los nuevos usuarios de la región de la UE
- Propósito de la prueba: probar cambios relacionados con la introducción de un sistema de recomendaciones mejorado
- Resultado esperado: dentro de los 14 días posteriores a la inscripción, los usuarios mostrarán una mejor conversión en vistas de la página del producto (el evento `product_page`), instancias de agregar artículos al carrito de compras (`product_card`) y compras (`purchase`). En cada etapa del embudo `product_page → product_card → purchase`, habrá al menos un 10% de aumento.
- Número previsto de participantes de la prueba: 6 000

In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.stats.proportion import proportions_ztest

In [2]:
# Leer los archivos con manejo adecuado de comillas y delimitadores
marketing = pd.read_csv('/ab_project_marketing_events_us_.txt')
events = pd.read_csv('/final_ab_events_upd_us.txt')
new_users = pd.read_csv('/final_ab_new_users_upd_us.txt')
participants = pd.read_csv('/final_ab_participants_upd_us.txt')

In [3]:
# Verificar la forma y mostrar información general de cada DataFrame
def explore_dataframe(df, name):
    print(f"\n{name} DataFrame:")
    print(f"Shape: {df.shape}")
    print("Head:\n", df.head())
    print("Info:")
    df.info()
    print("Describe:\n", df.describe(include='all'))
    print("Null values:\n", df.isnull().sum())
    print("Duplicate rows:", df.duplicated().sum())

explore_dataframe(marketing, "Marketing")
explore_dataframe(events, "Events")
explore_dataframe(new_users, "New Users")
explore_dataframe(participants, "Participants")



Marketing DataFrame:
Shape: (14, 4)
Head:
                            name                   regions    start_dt  \
0      Christmas&New Year Promo             EU, N.America  2020-12-25   
1  St. Valentine's Day Giveaway  EU, CIS, APAC, N.America  2020-02-14   
2        St. Patric's Day Promo             EU, N.America  2020-03-17   
3                  Easter Promo  EU, CIS, APAC, N.America  2020-04-12   
4             4th of July Promo                 N.America  2020-07-04   

    finish_dt  
0  2021-01-03  
1  2020-02-16  
2  2020-03-19  
3  2020-04-19  
4  2020-07-11  
Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   name       14 non-null     object
 1   regions    14 non-null     object
 2   start_dt   14 non-null     object
 3   finish_dt  14 non-null     object
dtypes: object(4)
memory usage: 576.0+ bytes
Describe:
                        

In [4]:
# Convertir columnas a datetime
marketing['start_dt'] = pd.to_datetime(marketing['start_dt'])
marketing['finish_dt'] = pd.to_datetime(marketing['finish_dt'])
events['event_dt'] = pd.to_datetime(events['event_dt'])
new_users['first_date'] = pd.to_datetime(new_users['first_date'])

# Mostrar información actualizada
print("Marketing DataFrame:")
print(marketing.info())

print("\nEvents DataFrame:")
print(events.info())

print("\nNew Users DataFrame:")
print(new_users.info())

# Análisis de valores ausentes en la columna 'details' del DataFrame 'events'
print("Valores ausentes en 'details':", events['details'].isnull().sum())
print("Porcentaje de valores ausentes en 'details':", events['details'].isnull().mean() * 100)

Marketing DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   name       14 non-null     object        
 1   regions    14 non-null     object        
 2   start_dt   14 non-null     datetime64[ns]
 3   finish_dt  14 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 576.0+ bytes
None

Events DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 423761 entries, 0 to 423760
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   user_id     423761 non-null  object        
 1   event_dt    423761 non-null  datetime64[ns]
 2   event_name  423761 non-null  object        
 3   details     60314 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 12.9+ MB
None

New Users DataFrame:
<cla

Decisión Basada en el Análisis

Si los valores ausentes en details corresponden principalmente a eventos que no necesitan detalles adicionales (como login o product_page), podemos dejarlos como están.

Si los valores ausentes en details corresponden a eventos que deberían tener detalles (como purchase), entonces debemos decidir si imputar estos valores o eliminarlos.

In [5]:
# Análisis de la columna 'details' en relación con 'event_name'
missing_details = events[events['details'].isnull()]
missing_details_by_event = missing_details['event_name'].value_counts()

# Mostrar la distribución de los valores ausentes en 'details' por tipo de evento
print(missing_details_by_event)

# Mostrar la distribución de los tipos de eventos en general
event_distribution = events['event_name'].value_counts()
print(event_distribution)


event_name
login           182465
product_page    120862
product_cart     60120
Name: count, dtype: int64
event_name
login           182465
product_page    120862
purchase         60314
product_cart     60120
Name: count, dtype: int64


Interpretación

Todos los eventos de tipo login, product_page, y product_cart tienen valores ausentes en details, lo cual es razonable porque estos eventos probablemente no requieran detalles adicionales.

Los eventos de tipo purchase tienen valores presentes en details, lo cual también tiene sentido ya que estos eventos deben registrar el monto de la compra.

Decisión

Dado que los valores ausentes en details se corresponden con eventos que no requieren detalles adicionales, no es necesario imputar estos valores. Podemos dejarlos como están.

In [6]:
# Unir los datos de participantes con los eventos
merged_data = pd.merge(participants, events, on='user_id')

**Calcular el Número de Eventos por Usuario y Estudiar la Distribución entre los Grupos**

In [7]:
# Calcular el número de eventos por usuario
events_per_user = merged_data.groupby('user_id').size()

# Unir con los datos de participantes para ver la distribución por grupo
events_per_user = events_per_user.reset_index(name='num_events')
events_per_user = pd.merge(events_per_user, participants, on='user_id')

# Calcular la distribución del número de eventos por usuario por grupo
events_per_user_distribution = events_per_user.groupby('group')['num_events'].describe()

# Mostrar la distribución del número de eventos por usuario por grupo
print(events_per_user_distribution)


        count      mean       std  min  25%  50%   75%   max
group                                                       
A      8214.0  8.042367  5.091125  1.0  4.0  6.0  10.0  40.0
B      6311.0  7.638726  4.831492  1.0  4.0  6.0  10.0  40.0


En general, parece que la distribución del número de eventos por usuario está equilibrada entre las muestras.

In [8]:
# Filtrar los eventos relevantes
product_page_views = merged_data[merged_data['event_name'] == 'product_page']
product_cart_adds = merged_data[merged_data['event_name'] == 'product_cart']
purchases = merged_data[merged_data['event_name'] == 'purchase']

# Calcular las conversiones para cada grupo
conversion_rate_product_page = product_page_views.groupby('group')['user_id'].nunique() / participants.groupby('group')['user_id'].nunique() * 100
conversion_rate_product_cart = product_cart_adds.groupby('group')['user_id'].nunique() / participants.groupby('group')['user_id'].nunique() * 100
conversion_rate_purchases = purchases.groupby('group')['user_id'].nunique() / participants.groupby('group')['user_id'].nunique() * 100

# Mostrar las conversiones
print("Conversion Rate for Product Page Views:")
print(conversion_rate_product_page)
print("\nConversion Rate for Product Cart Adds:")
print(conversion_rate_product_cart)
print("\nConversion Rate for Purchases:")
print(conversion_rate_purchases)


Conversion Rate for Product Page Views:
group
A    66.141732
B    64.238517
Name: user_id, dtype: float64

Conversion Rate for Product Cart Adds:
group
A    31.534163
B    32.828364
Name: user_id, dtype: float64

Conversion Rate for Purchases:
group
A    34.061468
B    32.360999
Name: user_id, dtype: float64


Basado en las tasas de conversión observadas:

El nuevo sistema de recomendaciones (grupo B) no parece haber mejorado significativamente la tasa de conversión en ninguna de las etapas del embudo en comparación con el grupo de control (grupo A).

En particular, la tasa de conversión para compras es ligeramente menor en el grupo B, lo que sugiere que el nuevo sistema de recomendaciones no tuvo el efecto positivo esperado.

**¿Hay usuarios que están presentes en ambas muestras?**

In [14]:
# Unir los datos de participantes con los eventos
merged_data = pd.merge(participants, events, on='user_id')

# Verificar si hay usuarios en ambas muestras
overlap_users = participants.groupby('user_id')['group'].nunique()
overlap_users = overlap_users[overlap_users > 1]
print(f"Usuarios presentes en ambos grupos: {len(overlap_users)}")


Usuarios presentes en ambos grupos: 441


**¿Cómo se distribuye el número de eventos entre los días?**

In [15]:
# Añadir una columna de día a los eventos
events['day'] = events['event_dt'].dt.date

# Contar el número de eventos por día
events_per_day = events.groupby('day').size()

# Mostrar la distribución de eventos por día
print(events_per_day)


day
2020-12-07    11385
2020-12-08    12547
2020-12-09    12122
2020-12-10    14077
2020-12-11    13864
2020-12-12    17634
2020-12-13    20985
2020-12-14    26184
2020-12-15    23469
2020-12-16    20909
2020-12-17    21751
2020-12-18    22871
2020-12-19    24273
2020-12-20    26425
2020-12-21    32559
2020-12-22    29472
2020-12-23    26108
2020-12-24    19399
2020-12-26    14058
2020-12-27    12420
2020-12-28    11014
2020-12-29    10146
2020-12-30       89
dtype: int64


**¿Hay alguna peculiaridad en los datos que hay que tener en cuenta antes de iniciar la prueba A/B?**

In [16]:
# Verificar si hay patrones extraños en los datos de eventos
events_by_group_day = merged_data.groupby(['group', 'day']).size().unstack().transpose()
print("Eventos por grupo y día:")
print(events_by_group_day)

Eventos por grupo y día:
group          A     B
day                   
2020-12-07  1375  1402
2020-12-08  1493  1411
2020-12-09  1557  1522
2020-12-10  1638  1526
2020-12-11  1672  1473
2020-12-12  2173  1867
2020-12-13  2335  2143
2020-12-14  3582  2556
2020-12-15  3294  2355
2020-12-16  3179  2291
2020-12-17  3366  2297
2020-12-18  3496  2307
2020-12-19  3706  2464
2020-12-20  4040  2636
2020-12-21  5057  3454
2020-12-22  3970  2888
2020-12-23  3454  2654
2020-12-24  2730  2002
2020-12-26  1929  1437
2020-12-27  1802  1285
2020-12-28  1537  1182
2020-12-29  1306   975
2020-12-30    14     6


El solapamiento de usuarios (441 usuarios en ambos grupos) es significativo y podría sesgar los resultados. Se recomienda manejar estos usuarios de alguna manera, ya sea excluyéndolos del análisis o asignándolos a un solo grupo.

In [17]:
# Filtrar usuarios que no están presentes en ambos grupos
unique_users = participants.groupby('user_id')['group'].nunique()
unique_users = unique_users[unique_users == 1].index
clean_participants = participants[participants['user_id'].isin(unique_users)]

# Unir los datos de participantes limpios con los eventos
clean_merged_data = pd.merge(clean_participants, events, on='user_id')

# Verificar el número de usuarios después de la limpieza
print(f"Usuarios únicos después de la limpieza: {clean_merged_data['user_id'].nunique()}")


Usuarios únicos después de la limpieza: 13197


In [18]:
# Filtrar los eventos relevantes
product_page_views_clean = clean_merged_data[clean_merged_data['event_name'] == 'product_page']
product_cart_adds_clean = clean_merged_data[clean_merged_data['event_name'] == 'product_cart']
purchases_clean = clean_merged_data[clean_merged_data['event_name'] == 'purchase']

# Calcular las conversiones para cada grupo
conversion_rate_product_page_clean = product_page_views_clean.groupby('group')['user_id'].nunique() / clean_participants.groupby('group')['user_id'].nunique() * 100
conversion_rate_product_cart_clean = product_cart_adds_clean.groupby('group')['user_id'].nunique() / clean_participants.groupby('group')['user_id'].nunique() * 100
conversion_rate_purchases_clean = purchases_clean.groupby('group')['user_id'].nunique() / clean_participants.groupby('group')['user_id'].nunique() * 100

# Mostrar las conversiones
print("Conversion Rate for Product Page Views (Cleaned):")
print(conversion_rate_product_page_clean)
print("\nConversion Rate for Product Cart Adds (Cleaned):")
print(conversion_rate_product_cart_clean)
print("\nConversion Rate for Purchases (Cleaned):")
print(conversion_rate_purchases_clean)


Conversion Rate for Product Page Views (Cleaned):
group
A    66.420019
B    64.451770
Name: user_id, dtype: float64

Conversion Rate for Product Cart Adds (Cleaned):
group
A    31.709942
B    33.154060
Name: user_id, dtype: float64

Conversion Rate for Purchases (Cleaned):
group
A    34.373739
B    32.633588
Name: user_id, dtype: float64


**Evaluación de los Resultados de la Prueba A/B**

In [20]:
# Calcular el número de usuarios únicos en cada grupo
n_users_A = clean_participants[clean_participants['group'] == 'A']['user_id'].nunique()
n_users_B = clean_participants[clean_participants['group'] == 'B']['user_id'].nunique()

# Calcular el número de conversiones en cada grupo
n_conv_product_page_A = product_page_views_clean[product_page_views_clean['group'] == 'A']['user_id'].nunique()
n_conv_product_page_B = product_page_views_clean[product_page_views_clean['group'] == 'B']['user_id'].nunique()

n_conv_product_cart_A = product_cart_adds_clean[product_cart_adds_clean['group'] == 'A']['user_id'].nunique()
n_conv_product_cart_B = product_cart_adds_clean[product_cart_adds_clean['group'] == 'B']['user_id'].nunique()

n_conv_purchases_A = purchases_clean[purchases_clean['group'] == 'A']['user_id'].nunique()
n_conv_purchases_B = purchases_clean[purchases_clean['group'] == 'B']['user_id'].nunique()

# Crear una función para realizar la prueba z y calcular los p-valores
def z_test_proportions(count_A, n_A, count_B, n_B):
    count = np.array([count_A, count_B])
    nobs = np.array([n_A, n_B])
    z_stat, p_value = proportions_ztest(count, nobs)
    return z_stat, p_value

# Realizar la prueba z para cada etapa del embudo
z_stat_product_page, p_value_product_page = z_test_proportions(n_conv_product_page_A, n_users_A, n_conv_product_page_B, n_users_B)
z_stat_product_cart, p_value_product_cart = z_test_proportions(n_conv_product_cart_A, n_users_A, n_conv_product_cart_B, n_users_B)
z_stat_purchases, p_value_purchases = z_test_proportions(n_conv_purchases_A, n_users_A, n_conv_purchases_B, n_users_B)

# Mostrar los resultados
print("Prueba Z para Vistas de la Página del Producto:")
print(f"Z-stat: {z_stat_product_page}, p-value: {p_value_product_page}")

print("\nPrueba Z para Agregar al Carrito:")
print(f"Z-stat: {z_stat_product_cart}, p-value: {p_value_product_cart}")

print("\nPrueba Z para Compras:")
print(f"Z-stat: {z_stat_purchases}, p-value: {p_value_purchases}")


Prueba Z para Vistas de la Página del Producto:
Z-stat: 2.360133405556827, p-value: 0.018268364199200356

Prueba Z para Agregar al Carrito:
Z-stat: -1.7590192171268264, p-value: 0.07857424456419275

Prueba Z para Compras:
Z-stat: 2.098920425931528, p-value: 0.03582391593808816


**Conclusiones Finales**

**EDA (Análisis Exploratorio de Datos)**

Distribución de Eventos:

La distribución del número de eventos por usuario es equilibrada entre los grupos A y B.
Hay picos de actividad en ciertos días, posiblemente relacionados con eventos específicos o campañas de marketing.
Usuarios en Ambas Muestras:

Se encontraron 441 usuarios en ambos grupos, lo que puede sesgar los resultados. Estos usuarios fueron excluidos para limpiar los datos.

Valores Ausentes:

Los valores ausentes en la columna details fueron razonables y no requerían imputación.

**Resultados de la Prueba A/B**

Vistas de la Página del Producto:

La diferencia en las tasas de conversión es significativa, con el grupo A (control) mostrando una mayor tasa de conversión.

Agregar al Carrito:

No hay una diferencia estadísticamente significativa en las tasas de conversión entre los grupos A y B.

Compras:

La diferencia en las tasas de conversión es significativa, con el grupo A (control) mostrando una mayor tasa de conversión.